In [51]:
from openai import OpenAI
from pydantic import BaseModel,Field
from typing import Optional
import json

ollama_client=OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

model="llama3.2"
# model="deepseek-r1:1.5b"
# model="deepseek-r1:8b"
# model="gemma:2b"

message=[
    {"role": "system", "content": "You are Ai assistance for Meeting deteils extractor and Meeting managing."}
]

response=ollama_client.chat.completions.create(
    model=model,
    messages=message
)

In [53]:
class MeetingExtractor(BaseModel):
    """The Meeting information extractor"""
    meeting_description:str=Field(description="Description of Meeting information.")
    is_calender_event:bool=Field(description="This text describe the Meeting is calender event or not.")
    confidence_score:float=Field(description="Confidence level between 0 to 1")


def extract_Meeting_info(userInput:str)->MeetingExtractor:
    
    msg1=[
        {'role':'system','content':'Analyze the text for meeting event.'},
        {'role':'user','content':userInput}
    ]
    task1=ollama_client.beta.chat.completions.parse(
        model=model,
        messages=msg1,
        response_format=MeetingExtractor,
    )

    message.append(task1.choices[0].message)
    # return task1.model_dump()
    return task1.choices[0].message.parsed

In [54]:
class MeetingDetails(BaseModel):
    """The Meeting details find"""
    title:str=Field(description="The title of meeting.")
    date:str=Field(description="The Meeting date and time, use day dd-mm-yyyy format. Example Monday, 01-03-2025")
    duration_of_meeting:float=Field(description="Time duration for complete the meeting.")
    meeting_participated_members:list[str]=Field(description="All members who attent the meeting.")

def extract_Meeting_details(userInput:str)->MeetingDetails:
    
    msg2=[
        {'role':'system','content':'Extract the meeting title, date, time, day,duration. exmaple Monday,01-03-2025.'},
        {'role':'user','content':userInput}
    ]
    task2=ollama_client.beta.chat.completions.parse(
        model=model,
        messages=msg2,
        response_format=MeetingDetails
    )

    message.append(task2.choices[0].message)
    # return task2.model_dump()
    return task2.choices[0].message.parsed

In [55]:
class EventConfirmation(BaseModel):
    """Generate confirmation message"""
    confirmation_message: str = Field(description="Natural language confirmation message")
    calendar_link: Optional[str] = Field(description="Generated calendar link if applicable")

def extract_Meeting_confirmation(userInput:str)->EventConfirmation:
    
    msg3=[
        {'role':'system','content':'Generate a natural confirmation message for the event.'},
        {'role':'user','content':userInput}
    ]
    task3=ollama_client.beta.chat.completions.parse(
        model=model,
        messages=msg3,
        response_format=EventConfirmation
    )

    message.append(task3.choices[0].message)
    # return task3.model_dump()
    return task3.choices[0].message.parsed

In [56]:
class MeetingData(BaseModel):
    """Meeting data"""

    meeting_description:str
    is_calender_event:bool
    confidence_score:float
    title:str
    date:str
    duration_of_meeting:float
    meeting_participated_members:list[str]
    confirmation_message:str
    calendar_link:Optional[str]



def getData(prompt:str)->MeetingData:

    info=extract_Meeting_info(prompt)

    meeting_description:str=info.meeting_description
    is_calender_event:bool=info.is_calender_event
    confidence_score:float=info.confidence_score

    if confidence_score>0.6:

        details=extract_Meeting_details(prompt)
        title:str=details.title
        date:str=details.date
        duration_of_meeting:float=details.duration_of_meeting
        meeting_participated_members:list[str]=details.meeting_participated_members

        confirmation=extract_Meeting_confirmation(prompt)
        confirmation_message: str=confirmation.confirmation_message
        calendar_link: Optional[str]=confirmation.calendar_link

        print(meeting_description,"\n",is_calender_event,"\n",confidence_score,"\n",title,"\n",date,"\n",duration_of_meeting,"\n",meeting_participated_members,"\n",confirmation_message,"\n",calendar_link)


        msg_main=[
            {'role':'system','content':'Generate a natural confirmation message for the event.'},
            {'role':'user','content':prompt}
        ]
        task_main=ollama_client.beta.chat.completions.parse(
            model=model,
            messages=msg_main,
            response_format=MeetingData
        )
        
        return task_main.model_dump()

    else:
        print("this is not meeting.....")

In [61]:
dt=getData("Let's schedule a 1h team meeting 25-02-2025, Tuesday at 2pm with Alice and Bob to discuss the project roadmap.")
# getData("Let's schedule a 30-minute catch-up with my manager to discuss my performance review.")
# getData("Schedule a 45-minute meeting with the design team to review the progress on the new website project.")
dt

project road map discussion 
 True 
 0.9 
 Project Roadmap Discussion 
 Wednesday, 26-02-2025 
 1.0 
 ['Alice', 'Bob'] 
 Your team meeting is confirmed.
Date: 25-02-2025
Time: Tuesday, 2pm
Attendees: Alice, Bob
Event Title: Project Roadmap Discussion
Please look forward to the meeting. 
 https://example.com/event/123456789


{'id': 'chatcmpl-828',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': '{"meeting_description": "Project Roadmap Discussion", "is_calender_event": true,"confidence_score": 0,"title": "", "date": "25-02-2025","duration_of_meeting": 60, "meeting_participated_members": ["" , ""],"confirmation_message": "This meeting is confirmed and will be held on Tuesday, the 25th of February at 2pm. We look forward to discussing the project roadmap with you, Alice and Bob.", "calendar_link": "<https://www.example.com/"}',
    'refusal': None,
    'role': 'assistant',
    'audio': None,
    'function_call': None,
    'tool_calls': [],
    'parsed': {'meeting_description': 'Project Roadmap Discussion',
     'is_calender_event': True,
     'confidence_score': 0.0,
     'title': '',
     'date': '25-02-2025',
     'duration_of_meeting': 60.0,
     'meeting_participated_members': ['', ''],
     'confirmation_message': 'This meeting is confirmed and will b

In [74]:
jsn=json.loads(dt['choices'][0]['message']['content'])
jsn['meeting_description']

'Project Roadmap Discussion'